In [9]:
from scipy.optimize import minimize
import numpy as np
from scipy.linalg import block_diag
from l1 import l1
import cvxopt
from cvxopt import normal
def unfold_mat(A):
    n = A.shape[0]*2
    d = A.shape[1]*2
    B = np.zeros((n, d))
    for i in range(int(n/2)):
        for j in range(int(d/2)):
            curr = A[i, j]
            a = curr.real
            b = curr.imag
            B[2*i, 2*j] = a
            B[2*i, 2*j+1] = b
            B[2*i+1, 2*j] = -b
            B[2*i+1, 2*j+1] = a
    return B

def unfold_vec(x):
    d = x.shape[0]*2
    y = np.zeros((d, 1))
    for i in range(int(d/2)):
        curr = x[i]
        a = curr.real
        b = curr.imag
        y[2*i]= a
        y[2*i+1] = b
    return y
# give the 1,2 norm of a vector in 2d
def norm12(x):
    norm = 0
    for i in range(int(len(x)/2)):
        norm = norm + np.sqrt(x[2*i]**2 + x[2*i+1]**2)
    return norm
def gen_gaussian(k, t, n):
    block_list = []
    G = np.random.randn(k, t) / k*np.sqrt(np.pi/2)
    for i in range(n):
        
        block_list.append(G)
    return block_diag(*block_list)
n=2
d=1
A = np.zeros((n,d),dtype=np.complex_)
x = np.zeros((d,1),dtype=np.complex_)
y = np.zeros((n,1),dtype=np.complex_)

for i in range(n):
    for j in range(d):
        a = np.random.randint(0, 10)
        b = np.random.randint(0, 10)
        A[i,j]=complex(a,b)

for i in range(d):
    a = np.random.randint(0, 10)
    b = np.random.randint(0, 10)
    x[i] = complex(a,b)
    
exact_res = 0
for i in range(n):
    a = np.random.randint(0, 10)
    b = np.random.randint(0, 10)
    exact_res = exact_res + np.sqrt(a**2+b**2)
    y[i] = np.vdot(A[i,:], x) + complex(a,b)


def fit(X, params):
    return np.dot(np.conj(X), params)

def cost_function(params, X, y):
#    return np.linalg.norm(y - fit(X, params.reshape((len(params), 1))), ord=1)
    return norm12(y - fit(X, params.reshape((len(params), 1))))[0]

AA = unfold_mat(A)
yy = unfold_vec(y)
# print(fit(A, x))
# x0 = np.linalg.lstsq(AA,yy)[0]
x0=np.zeros((2*d, 1))
# print(cost_function(x, A, y))
print(exact_res)

k=2
G = gen_gaussian(k, 2, n)
AAA = cvxopt.matrix(np.dot(G, AA))
yyy = cvxopt.matrix(np.dot(G, yy))
x1 = l1(AAA, yyy)
y1 = np.dot(AA, x1)
print("cvxopt residual: " + str(norm12(y1-yy)))
print("cvxopt output: ")
print(x1)
output = minimize(cost_function, x0, args=(AA, yy))

y_hat = fit(AA, output.x.reshape(2*d, 1))
print(output.x)
print("scipy residual: " + str(norm12(y_hat-yy)))
#print("scipy residual: " + str(np.linalg.norm(y_hat-y)))
print(A)
print(x)
print(y)
print(AA)
print(yy)

9.06225774829855
cvxopt residual: [4.26383806]
cvxopt output: 
[ 3.44e+00]
[ 1.64e+00]

[3.79310345 1.48275862]
scipy residual: [1.71202644]
[[2.+5.j]
 [1.+1.j]]
[[4.+0.j]]
[[15.-16.j]
 [ 5. -4.j]]
[[ 2.  5.]
 [-5.  2.]
 [ 1.  1.]
 [-1.  1.]]
[[ 15.]
 [-16.]
 [  5.]
 [ -4.]]
